In [1]:
import glob
import os

In [2]:
if os.path.isfile('../data/all_guides.bed'):
    ! rm ../data/all_guides.bed

guide_fns = glob.glob('../data/*guides.bed')
print("Make sure PanGATA is in this list: ",guide_fns)

all_files = " ".join(guide_fns)
! cat $all_files > ../data/all_guides.bed
! cut -f1,2,3 ../data/all_guides.bed | sort -k1,1 -k2,2n | mergeBed -d 1000000 -i - > ../data/merged_target_area__hg38.bed
! ~/ucsc_tools/liftOver ../data/merged_target_area__hg38.bed ../data/external/hg38ToHg19.over.chain.gz ../data/merged_target_area__hg19.bed ../data/merged_target_area__unMapped.bed
! bedtools slop -i ../data/merged_target_area__hg19.bed -g ../data/external/hg19.chrom.sizes -b 500000 | mergeBed -d 1000000 -i - > ../data/merged_target_filter__hg19.bed

Make sure PanGATA is in this list:  ['../data/MEF2C_guides.bed', '../data/NMU_guides.bed', '../data/PanGATA1_hg38_padded_guides.bed', '../data/CD164_guides.bed', '../data/LMO2_guides.bed', '../data/ATXN_guides.bed', '../data/FADS_hg38_guides.bed']
Reading liftover chains
Mapping coordinates


In [3]:
with open('../data/merged_target_filter__hg19.bed', 'r') as f:
    interval_list = [ [ int(field) if idx>0 else field 
                        for idx, field in enumerate(line.rstrip().split()) 
                      ] 
                     for line in f.readlines() 
                    ]

for strand in ['minus','plus']:
    hg19_bw = f'../data/external/GSM1480327_K562_PROseq_{strand}.bw'
    hg19_bg = f'../data/external/merged_targets__hg19__K562_PROseq_{strand}.bedGraph'
    hg38_bg = f'../data/external/merged_targets__hg38__K562_PROseq_{strand}.bedGraph'
    unMap_fn= f'../data/external/merged_targets__unMapped__K562_PROseq_{strand}.bedGraph'
    hg38_bw = f'../data/external/merged_targets__hg38__K562_PROseq_{strand}.bw'
    hg38_wig= f'../data/external/merged_targets__hg38__K562_PROseq_{strand}.wig'
    top_str = f"track type=wiggle_0 name=PROseq_{strand}_strand"
    temp_files = []
    for idx, interval in enumerate(interval_list):
        tmp_fn = f'../data/external/tmp_{idx}.bedGraph'
        temp_files.append(tmp_fn)
        chrom, start, end = interval
        ! ~/ucsc_tools/bigWigToBedGraph -chrom=$chrom $hg19_bw $tmp_fn
    print(f"made BedGraphs from {hg19_bw}")
    pack_temp = ' '.join(temp_files)
    ! cat $pack_temp > trash.txt
    ! sort -k1,1 -k2,2n trash.txt | intersectBed -u -a - -b ../data/merged_target_filter__hg19.bed > $hg19_bg
    ! rm $pack_temp
    ! rm trash.txt
    print(f"cat and sort BedGraphs into {hg19_bg}")
    ! ~/ucsc_tools/liftOver $hg19_bg ../data/external/hg19ToHg38.over.chain.gz trash.txt $unMap_fn 
    ! sort -k1,1 -k2,2n trash.txt | mergeBed -d -1 -c 4 -o mean -i - > $hg38_bg
    ! rm trash.txt
    print(f"Lifted {hg19_bg} to {hg38_bg}")
    ! ~/ucsc_tools/bedGraphToBigWig $hg38_bg ../data/external/hg38.chrom.sizes $hg38_bw 
    print(f"Dumped to BigWig {hg38_bw}")
    ! ~/ucsc_tools/bigWigToWig $hg38_bw $hg38_wig 
    print(f"Dumped to wiggle {hg38_wig}")
    ! echo $top_str | cat - $hg38_wig > /tmp/out && mv /tmp/out $hg38_wig


made BedGraphs from ../data/external/GSM1480327_K562_PROseq_minus.bw
cat and sort BedGraphs into ../data/external/merged_targets__hg19__K562_PROseq_minus.bedGraph
Reading liftover chains
Mapping coordinates
Lifted ../data/external/merged_targets__hg19__K562_PROseq_minus.bedGraph to ../data/external/merged_targets__hg38__K562_PROseq_minus.bedGraph
Dumped to BigWig ../data/external/merged_targets__hg38__K562_PROseq_minus.bw
Dumped to wiggle ../data/external/merged_targets__hg38__K562_PROseq_minus.wig
made BedGraphs from ../data/external/GSM1480327_K562_PROseq_plus.bw
cat and sort BedGraphs into ../data/external/merged_targets__hg19__K562_PROseq_plus.bedGraph
Reading liftover chains
Mapping coordinates
Lifted ../data/external/merged_targets__hg19__K562_PROseq_plus.bedGraph to ../data/external/merged_targets__hg38__K562_PROseq_plus.bedGraph
Dumped to BigWig ../data/external/merged_targets__hg38__K562_PROseq_plus.bw
Dumped to wiggle ../data/external/merged_targets__hg38__K562_PROseq_plus.wi